In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
df = pd.read_csv('dataset_cleaned_v3.csv')
df = df[df['app_desc'].notna()]
df

,id,track_name,price,cont_rating,prime_genre,vpp_lic,app_desc
0,281656475,PAC-MAN Premium,3.99,4,games,1,save limited popular video arcade history worl...
1,281796108,Evernote - stay organized,0.00,4,productivity,1,let change way organize personal professional ...
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",0.00,4,weather,1,popular free weather powered professional weat...
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",0.00,12,shopping,1,best way find anything need fashion new used b...
4,282935706,Bible,0.00,4,reference,1,million around world people reading listening ...
...,...,...,...,...,...,...,...
6918,1187617475,Kubik,0.00,4,games,1,place falling correctly order clear left right...
6919,1187682390,VR Roller-Coaster,0.00,4,games,1,thrilling virtual reality roller coaster exper...
6920,1187779532,Bret Michaels Emojis + Lyric Keyboard,1.99,9,utilities,1,rock star bret winner celebrity apprentice cha...
6921,1187838770,VR Roller Coaster World - Virtual Reality,0.00,12,games,1,roller coaster world cardboard veer best rolle...


In [20]:
genres = set(df['prime_genre'].to_list())
genres

{'book',
 'business',
 'catalogs',
 'education',
 'entertainment',
 'finance',
 'food & drink',
 'games',
 'health & fitness',
 'lifestyle',
 'medical',
 'music',
 'navigation',
 'news',
 'photo & video',
 'productivity',
 'reference',
 'shopping',
 'social networking',
 'sports',
 'travel',
 'utilities',
 'weather'}

In [21]:
df_genre = df.groupby(df['prime_genre'])

In [22]:
# grouping dataset
dict_apps = {}
for genre in genres:
    dict_apps[genre] = {}
    dict_apps[genre]['data'] = df_genre.get_group(genre)
    corpus = []
    vectorizer = TfidfVectorizer()
    for row in dict_apps[genre]['data'].iterrows():
        corpus.append(row[1].app_desc)
    dict_apps[genre]['corpus'] = corpus
    vectorizer.fit_transform(corpus)
    indices = np.argsort(vectorizer.idf_)[::-1]
    features = vectorizer.get_feature_names()
    top_n = 3
    top_features = [str(features[i]) for i in indices[:top_n]]
    dict_apps[genre]['top_word'] = ' '.join(top_features)

In [23]:
grouping_df = pd.DataFrame(columns=['prime_genre','total','top 3'])
for key in dict_apps.keys():
    grouping_df=grouping_df.append({'prime_genre': key, 'total': len(dict_apps[key]['data']), 'top 3': dict_apps[key]['top_word']}, ignore_index=True)

In [24]:
grouping_df

,prime_genre,total,top 3
0,music,138,abandon musicality mixture
1,book,81,zoom half han
2,shopping,102,zoom dressed distance
3,news,69,zoom domestic passionate
4,lifestyle,119,zoom imagination horizontal
5,food & drink,55,zip item introduction
6,sports,111,license partial pat
7,medical,23,lung prevent must
8,photo & video,347,cross reminder cliche
9,travel,71,zone install importantly


In [25]:
grouping_df.to_csv('total_masing2.csv')